## Data Manipulation

In [1]:
# loads libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from scipy.stats import norm, skew #for some statistics
import scipy.special



%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns # nice plotting
color = sns.color_palette() # setting color palette
sns.set_style('darkgrid') # setting a dark grid background

# something about warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
#pd.options.mode.chained_assignment = None


# Machine learning libraries
#import keras (This is only commented out because Aviv couldn't install keras on his conda.  He will fix this eventually)
import sklearn
from sklearn.preprocessing import LabelEncoder

In [2]:
# changes display options to allow many rows and columns to be displayed
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [3]:
# uploads data sets
train = pd.read_csv("../Data/House Prices Advanced Regression Techniques/train.csv")
test = pd.read_csv("../Data/House Prices Advanced Regression Techniques/test.csv")
total = pd.concat([train, test], axis=0)

# makes duplicate data sets for train data
total2 = total

In [4]:
total2

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,Id,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SalePrice,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,NaN,3,1Fam,TA,No,706.000,0.000,GLQ,Unf,1.000,0.000,Gd,150.000,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,NaN,0,PConc,2,Typ,548.000,2.000,TA,RFn,TA,Attchd,2003.000,1710,1,GasA,Ex,2Story,1,1,Gd,Lvl,Gtl,8450,Inside,65.000,Reg,0,60,RL,196.000,BrkFace,NaN,0,2,CollgCr,61,5,7,Y,0,NaN,CompShg,Gable,Normal,208500.000,WD,0,Pave,8,856.000,AllPub,0,2003,2003,2008
1,1262,0,0,NaN,3,1Fam,TA,Gd,978.000,0.000,ALQ,Unf,0.000,1.000,Gd,284.000,Y,Feedr,Norm,SBrkr,0,TA,TA,MetalSd,MetalSd,NaN,TA,1,CBlock,2,Typ,460.000,2.000,TA,RFn,TA,Attchd,1976.000,1262,0,GasA,Ex,1Story,2,1,TA,Lvl,Gtl,9600,FR2,80.000,Reg,0,20,RL,0.000,None,NaN,0,5,Veenker,0,8,6,Y,0,NaN,CompShg,Gable,Normal,181500.000,WD,0,Pave,6,1262.000,AllPub,298,1976,1976,2007
2,920,866,0,NaN,3,1Fam,TA,Mn,486.000,0.000,GLQ,Unf,1.000,0.000,Gd,434.000,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,TA,1,PConc,2,Typ,608.000,2.000,TA,RFn,TA,Attchd,2001.000,1786,1,GasA,Ex,2Story,3,1,Gd,Lvl,Gtl,11250,Inside,68.000,IR1,0,60,RL,162.000,BrkFace,NaN,0,9,CollgCr,42,5,7,Y,0,NaN,CompShg,Gable,Normal,223500.000,WD,0,Pave,6,920.000,AllPub,0,2001,2002,2008
3,961,756,0,NaN,3,1Fam,Gd,No,216.000,0.000,ALQ,Unf,1.000,0.000,TA,540.000,Y,Norm,Norm,SBrkr,272,TA,TA,Wd Sdng,Wd Shng,NaN,Gd,1,BrkTil,1,Typ,642.000,3.000,TA,Unf,TA,Detchd,1998.000,1717,0,GasA,Gd,2Story,4,1,Gd,Lvl,Gtl,9550,Corner,60.000,IR1,0,70,RL,0.000,None,NaN,0,2,Crawfor,35,5,7,Y,0,NaN,CompShg,Gable,Abnorml,140000.000,WD,0,Pave,7,756.000,AllPub,0,1915,1970,2006
4,1145,1053,0,NaN,4,1Fam,TA,Av,655.000,0.000,GLQ,Unf,1.000,0.000,Gd,490.000,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,TA,1,PConc,2,Typ,836.000,3.000,TA,RFn,TA,Attchd,2000.000,2198,1,GasA,Ex,2Story,5,1,Gd,Lvl,Gtl,14260,FR2,84.000,IR1,0,60,RL,350.000,BrkFace,NaN,0,12,NoRidge,84,5,8,Y,0,NaN,CompShg,Gable,Normal,250000.000,WD,0,Pave,9,1145.000,AllPub,192,2000,2000,2008
5,796,566,320,NaN,1,1Fam,TA,No,732.000,0.000,GLQ,Unf,1.000,0.000,Gd,64.000,Y,Norm,Norm,SBrkr,0,TA,TA,VinylSd,VinylSd,MnPrv,NaN,0,Wood,1,Typ,480.000,2.000,TA,Unf,TA,Attchd,1993.000,1362,1,GasA,Ex,1.5Fin,6,1,TA,Lvl,Gtl,14115,Inside,85.000,IR1,0,50,RL,0.000,None,Shed,700,10,Mitchel,30,5,5,Y,0,NaN,CompShg,Gable,Normal,143000.000,WD,0,Pave,5,796.000,AllPub,40,1993,1995,2009
6,1694,0,0,NaN,3,1Fam,TA,Av,1369.000,0.000,GLQ,Unf,1.000,0.000,Ex,317.000,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,Gd,1,PConc,2,Typ,636.000,2.000,TA,RFn,TA,Attchd,2004.000,1694,0,GasA,Ex,1Story,7,1,Gd,Lvl,Gtl,10084,Inside,75.000,Reg,0,20,RL,186.000,Stone,NaN,0,8,Somerst,57,5,8,Y,0,NaN,CompShg,Gable,Normal,307000.000,WD,0,Pave,7,1686.000,AllPub,255,2004,2005,2007
7,1107,983,0,NaN,3,1Fam,TA,Mn,859.000,32.000,ALQ,BLQ,1.000,0.000,Gd,216.000,Y,PosN,Norm,SBrkr,228,TA,TA,HdBoard,HdBoard,NaN,TA,2,CBlock,2,Typ,484.000,2.000,TA,RFn,TA,Attchd,1973.000,2090,1,GasA,Ex,2Story,8,1,TA,Lvl,Gtl,10382,Corner,nan,IR1,0,60,RL,240.000,Stone,Shed,350,11,NWAmes,204,6,7,Y,0,NaN,CompShg,Gable,Normal,200000.000,WD,0,Pave,7,1107.000,AllPub,235,1973,1973,2009
8,1022,752,0,NaN,2,1Fam,TA,No,0.000,0.000,Unf,Unf,0.000,0.000,TA,952.000,Y,Artery,Norm,FuseF,205,TA,TA,BrkFace,Wd Shng,NaN,TA,2,BrkTil,2,Min1,468.000,2.000,TA,Unf,Fa,Detchd,1931.000,1774

In [18]:
total2.shape

(2919, 83)

In [19]:
train.shape

(1460, 81)

In [24]:
train.shape[1]

81

In [20]:
test.shape

(1459, 80)

In [6]:
# makes an indicator variable for whether there was LotFrontage was NA
cond = total2['LotFrontage'].isnull()
total2['HasLotFrontage'] = cond * 1

# gets LotFrontage from sqrt(LotArea) for a new LotFrontage variable
total2['LotFrontage2'] = total2['LotFrontage']
sqrtlotarea = np.sqrt(total2['LotArea'])
total2['LotFrontage2'][cond] = sqrtlotarea[cond]

# changes NaN's in GarageType to "None" and imputes yearbuilt for garageyrblt for houses with no garages
cond = total2['GarageType'].isnull()
total2['GarageType'][cond] = "None"
total2['GarageYrBlt'][cond] = total2['YearBuilt'][cond]

# imputes -1 for missing values in MasVnrArea
total2.MasVnrArea = total2.MasVnrArea.fillna(-1)

In [9]:
# removes changes null values that shouldn't be treated as null
total2['Alley'] = total2['Alley'].fillna("None")
total2['BsmtQual'] = total2['BsmtQual'].fillna("None")
total2['BsmtCond'] = total2['BsmtCond'].fillna("None")
total2['BsmtExposure'] = total2['BsmtExposure'].fillna("None")
total2['BsmtFinType1'] = total2['BsmtFinType1'].fillna("None")
total2['BsmtFinType2'] = total2['BsmtFinType2'].fillna("None")
total2['FireplaceQu'] = total2['FireplaceQu'].fillna("None")
total2['GarageType'] = total2['GarageType'].fillna("None")
total2['GarageFinish'] = total2['GarageFinish'].fillna("None")
total2['GarageQual'] = total2['GarageQual'].fillna("None")
total2['GarageCond'] = total2['GarageCond'].fillna("None")
total2['PoolQC'] = total2['PoolQC'].fillna("None")
total2['Fence'] = total2['Fence'].fillna("None")
total2['MiscFeature'] = total2['MiscFeature'].fillna("None")
total2['MasVnrType'] = total2['MasVnrType'].fillna("missing")

In [10]:
# recodes some categorical variables as ordinal
total2 = total2.replace({"Alley" : {"None" : 0, "Grvl" : 1, "Pave" : 2},
                       "BsmtCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "BsmtExposure" : {"None" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
                       "BsmtFinType1" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2" : {"None" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
                       "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
                       "FireplaceQu" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, 
                                       "Min2" : 6, "Min1" : 7, "Typ" : 8},
                       "GarageCond" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "GarageFinish" : {"None" : 0, "Unf" : 1, "RFn" : 2, "Fin" : 3},
                       "GarageQual" : {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
                       "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
                       "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC" : {"None" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
                       "Street" : {"Grvl" : 1, "Pave" : 2},
                       "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}})

In [11]:
# fixing specific na cells
total2.BsmtFinSF1 = total2.BsmtFinSF1.fillna(0)
total2.BsmtFinSF2 = total2.BsmtFinSF2.fillna(0)
total2.BsmtFullBath = total2.BsmtFullBath.fillna(0)
total2.BsmtHalfBath = total2.BsmtHalfBath.fillna(0)
total2.BsmtUnfSF = total2.BsmtUnfSF.fillna(0)

In [12]:
# removes changes null values that shouldn't be treated as null
total2['Alley'] = total2['Alley'].fillna("None")
total2['BsmtQual'] = total2['BsmtQual'].fillna("None")
total2['BsmtCond'] = total2['BsmtCond'].fillna("None")
total2['BsmtExposure'] = total2['BsmtExposure'].fillna("None")
total2['BsmtFinType1'] = total2['BsmtFinType1'].fillna("None")
total2['BsmtFinType2'] = total2['BsmtFinType2'].fillna("None")
total2['FireplaceQu'] = total2['FireplaceQu'].fillna("None")
total2['GarageType'] = total2['GarageType'].fillna("None")
total2['GarageFinish'] = total2['GarageFinish'].fillna("None")
total2['GarageQual'] = total2['GarageQual'].fillna("None")
total2['GarageCond'] = total2['GarageCond'].fillna("None")
total2['PoolQC'] = total2['PoolQC'].fillna("None")
total2['Fence'] = total2['Fence'].fillna("None")
total2['MiscFeature'] = total2['MiscFeature'].fillna("None")

In [13]:
total2 = total2.fillna(total2.mean())

In [14]:
#what does this do?
total2 = total2.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [15]:
#converting Sales Price to log scale
total2.SalePrice = np.log(total2.SalePrice)

In [16]:
# Checking whether there is any null values
total2.isnull().sum()

1stFlrSF          0
2ndFlrSF          0
3SsnPorch         0
Alley             0
BedroomAbvGr      0
BldgType          0
BsmtCond          0
BsmtExposure      0
BsmtFinSF1        0
BsmtFinSF2        0
BsmtFinType1      0
BsmtFinType2      0
BsmtFullBath      0
BsmtHalfBath      0
BsmtQual          0
BsmtUnfSF         0
CentralAir        0
Condition1        0
Condition2        0
Electrical        0
EnclosedPorch     0
ExterCond         0
ExterQual         0
Exterior1st       0
Exterior2nd       0
Fence             0
FireplaceQu       0
Fireplaces        0
Foundation        0
FullBath          0
Functional        0
GarageArea        0
GarageCars        0
GarageCond        0
GarageFinish      0
GarageQual        0
GarageType        0
GarageYrBlt       0
GrLivArea         0
HalfBath          0
Heating           0
HeatingQC         0
HouseStyle        0
Id                0
KitchenAbvGr      0
KitchenQual       0
LandContour       0
LandSlope         0
LotArea           0
LotConfig         0
